<a href="https://colab.research.google.com/github/vemuriPradyumna/Chatbot_nltk/blob/main/Chatbot_nltk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from tensorflow.keras.optimizers import SGD
import random
import numpy as np
import json
import pickle
from google.colab import files
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
lem = WordNetLemmatizer()
words, classes, documents, ignore_words = list(), list(), list(), ['!','?'] 

In [ ]:
%cd /content

/content


In [ ]:
from google.colab import drive
 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
intents = json.loads(open("gdrive/MyDrive/Colab/Chatbot_nltk/intents.json").read())

In [ ]:
intents

{'intents': [{'context': [''],
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'context': [''],
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'context': [''],
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'context': [''],
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'tag': 'noanswer'},
  {'context': [''],
   'patterns': ['How you could help me?',
    'What you can do?

In [ ]:
from nltk.tokenize import word_tokenize
for i in intents['intents']:
  for pattern in i['patterns']:

    word = nltk.word_tokenize(pattern)
    words.extend(word)
    documents.append([word, i['tag']])

    if i['tag'] not in classes:
      classes.append(i['tag'])

In [ ]:
words = [lem.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print(len(documents))
print(len(classes)," classes of clauses")
print(len(words), " Lematized words")

47
9  classes of clauses
88  Lematized words


In [ ]:
pickle.dump(words,open("gdrive/MyDrive/Colab/Chatbot_nltk/words.pickle","wb"))
pickle.dump(classes,open("gdrive/MyDrive/Colab/Chatbot_nltk/classes.pickle","wb"))
pickle.dump(documents,open("gdrive/MyDrive/Colab/Chatbot_nltk/documents.pickle","wb"))

In [ ]:
classes

['adverse_drug',
 'blood_pressure',
 'blood_pressure_search',
 'goodbye',
 'greeting',
 'hospital_search',
 'options',
 'pharmacy_search',
 'thanks']

In [ ]:
training = list()
output_classes = np.zeros(len(classes),np.int8)
for doc in documents:
  bag_of_words = list()
  pattern_words = doc[0]
  pattern_words = [ lem.lemmatize(w.lower()) for w in pattern_words]
  for w in words:
    if w in pattern_words:
      bag_of_words.append(1)
    else:
      bag_of_words.append(0)
  
  output_row = list(output_classes)
  output_row[classes.index(doc[1])] = 1

  training.append([bag_of_words,output_row])

random.shuffle(training)
training = np.array(training)  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
X = list(training[:,0])
Y = list(training[:,1])

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(X[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(Y[0]), activation='softmax'))

In [ ]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
h = model.fit(x=np.array(X,dtype=np.float),y=np.array(Y,dtype=np.float),epochs=200,batch_size=5,verbose=1)

Epoch 1/200
10/10 [==============================] - 1s 3ms/step - loss: 2.2405 - accuracy: 0.1064
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1753 - accuracy: 0.1702
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0811 - accuracy: 0.1702
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9463 - accuracy: 0.3830
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8685 - accuracy: 0.4043
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8854 - accuracy: 0.3404
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.6558 - accuracy: 0.5106
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.5880 - accuracy: 0.4468
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3071 - accuracy: 0.7021
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3795 - accuracy: 0.6383
Epoch 11/

In [ ]:
model.save('gdrive/MyDrive/Colab/Chatbot_nltk/model.h5',h)